描述符是对多个属性运用存取逻辑的一种方式。

描述符是实现了特定协议的类，这个协议包括 \_\_get__、\_\_set__、\_\_delete__ 方法。property 类实现了完整的描述符协议。通常，可以只实现部分协议。

定义
- 描述符类
    - 实现描述符协议的类
- 托管类
    - 把描述符实例声明为类属性的类
- 描述符实例
    - 描述符类的各个实例，声明为托管类的类属性
- 托管实例
    - 托管类的实例
- 存储属性
    - 托管实例中存储自身托管属性的属性
- 托管属性
    - 托管类中由描述符实例处理的公开属性，值存储在存储属性中

描述符的用法是，创建一个实例，作为另一个类的类属性。

![](20-1.png)

管理实例属性的描述符应该把值存储在托管实例中

In [1]:
class Quantity:  # 描述符基于协议实现，无需创建子类

    def __init__(self, storage_name):
        self.storage_name = storage_name  # 托管实例中存储值的属性的名称

    # 描述符实例、托管实例、要设定的值
    def __set__(self, instance, value):  # 尝试为托管属性赋值时，会调用 __set___ 方法
        if value > 0:
            instance.__dict__[self.storage_name] = value  # 处理托管实例的 __dict__ 属性
        else:
            raise ValueError('value must be > 0')

            
# 使用描述符
class LineItem:
    weight = Quantity('weight')  # 绑定描述符实例
    price = Quantity('price')  # 绑定描述符实例

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [2]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

nutmeg.\_Quantity#0 是无效的 Python 句法，但是内置的 getattr 和 setattr 函数可以使用这种“无效的”标识符获取和设置属性，此外也可以直接处理实例属性 \_\_dict__ 。

![](20-4.png)

这里可以使用内置的高阶函数 getattr 和 setattr 存取值，因为托管属性和存储属性的名称不同。

\_\_get__ 方法有三个参数：self、instance、owner
- owner 参数是托管类的引用，通过描述符从托管类中获取属性时用得到

In [3]:
# 自动获取存储属性的名称
class Quantity:
    __counter = 0  # 类属性，统计 Quantity 实例数量

    def __init__(self):
        cls = self.__class__  # Quantity 类的引用
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)  # 每个描述符实现的 storage_name 属性都是独一无二的
        cls.__counter += 1  # 递增

    def __get__(self, instance, owner):  # 托管属性名称与 storage_name 不同
        return getattr(instance, self.storage_name)  # 获取存储属性的值

    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)  # 把值存在 instance 中
        else:
            raise ValueError('value must be > 0')


class LineItem:
    weight = Quantity()  # 不用把托管属性的名称传递给 Quantity 构造方法
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [4]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)
coconuts.weight, coconuts.price

(20, 17.95)

In [5]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

(20, 17.95)

In [6]:
LineItem.weight

AttributeError: 'NoneType' object has no attribute '_Quantity#0'

为了给用户提供内省和洽谈元编程技术支持，通过类访问托管属性时，最好让 \_\_get__ 方法返回描述符实例。

In [7]:
class Quantity:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self  # 如果不是通过实例调用，返回描述符本身
        else:
            return getattr(instance, self.storage_name)  # 返回托管属性的值

    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')


class LineItem:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [8]:
LineItem.price

In [9]:
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

通常，我们不会再使用描述符的模块中定义描述符，而是在一个单独的实用工具模块中定义，以便在整个应用中使用。如果开发的是框架，甚至会在多个应用中使用。

如果想自动把存储属性的名称设置成与托管属性的名称类似，需要用到类装饰器或元类。

In [10]:
import model_v4c as model  # 导入模块


class LineItem:
    weight = model.Quantity()  # 使用描述符
    price = model.Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

相比于特性工厂函数，描述符类：
- 可以使用子类扩展
- 在类属性和实例属性中保持状态更易于理解（下面的特性工厂函数使用函数属性和闭包）

In [11]:
# 特性工厂函数
def quantity():  # 没有 storage_name 参数
    try:
        quantity.counter += 1  # 不能依靠类属性在多次调用之间共享，因此定义为 quantity 函数的属性
    except AttributeError:
        quantity.counter = 0  # 属性初始化为 0

    storage_name = '_{}:{}'.format('quantity', quantity.counter)  # 创建局部变量，借助闭包保持

    def qty_getter(instance):  # 使用内置的 getattr
        return getattr(instance, storage_name)

    def qty_setter(instance, value):  # 使用内置的 setattr
        if value > 0:
            setattr(instance, storage_name, value)
        else:
            raise ValueError('value must be > 0')

    return property(qty_getter, qty_setter)


class LineItem:
    weight = quantity()
    price = quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

Validated、Quantity、NonBlank 三个类之间的关系体现了模板方法设计模式：一个模板方法用一些抽象的操作定义一个算法，而子类将重定义这些操作以提供具体的行为。

![](20-5.png)

In [12]:
import abc


# 描述符类
class AutoStorage: 
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)


# 抽象类
class Validated(abc.ABC, AutoStorage):

    def __set__(self, instance, value):
        value = self.validate(instance, value)  # 把验证委托给 validate 方法
        super().__set__(instance, value)

    @abc.abstractmethod  # 抽象方法
    def validate(self, instance, value):
        """return validated value or raise ValueError"""


# 继承 Validated
class Quantity(Validated):
    """a number greater than zero"""

    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value


# 继承 Validated
class NonBlank(Validated):
    """a string with at least one non-space character"""

    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value  # 返回验证后的值

In [13]:
import model_v5 as model  # <1>


class LineItem:
    description = model.NonBlank()  # <2>
    weight = model.Quantity()
    price = model.Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [14]:
void = LineItem('', 1, 1)

ValueError: value cannot be empty or blank

上面买的 LineItem 示例演示了描述符的典型用途 —— 管理数据属性。这种描述符也叫覆盖性描述符，因为描述符的 \_\_set__ 方法使用托管实例中的同名属性覆盖（即插手接管）了要设置的属性。

通过实例读取属性时，通常返回的是实例中定义的属性；但是，如果实例中没有指定的属性，那么会获取类属性。而为实例中的属性赋值时，通常会在实例中创建实例，根本不影响类。

In [15]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))


# 有 __get__ 和 __set__ 方法的典型覆盖型描述符
class Overriding:  # <1>
    """也称数据描述符或强制描述符"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

    def __set__(self, instance, value):
        print_args('set', self, instance, value)

        
# 没有 __get__ 方法的覆盖型描述符
class OverridingNoGet:
    """没有 __get__ 方法的覆盖型描述符"""

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


# 没有 __set__ 方法，所以是非覆盖型描述符
class NonOverriding:
    """也称非数据描述符或覆盖型描述符"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

# 托管类，使用各个描述符类
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):  # 方法也是描述符
        print('-> Managed.spam({})'.format(display(self)))

In [16]:
obj = Managed()  # 测试用的对象
obj.over  # 触发描述符的 __get__ 方法，第二个参数的值是托管实例 obj

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [17]:
Managed.over  # 触发描述符的 __get__ 方法，第二个参数（instance）的值是 None

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [18]:
obj.over = 7  # 触发描述符的 __set__ 方法，最后一个参数的值是 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [19]:
obj.over  # 触发描述符的 __get__ 方法

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [20]:
obj.__dict__['over'] = 8  # 跳过描述符，通过 obj.__dict__ 属性设值
vars(obj)  # 确认值在 obj.__dict__ 属性中，在 over 键下

{'over': 8}

In [21]:
obj.over  # Managed.over 描述符仍会覆盖读取 obj.over 的操作

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


覆盖型描述符可以只实现 \_\_set__ 方法，实例属性会覆盖描述符，不过只有读操作会。

In [22]:
obj.over_no_get  # 这个描述符没有 __get__ 方法，从类中获取描述符实例

In [23]:
Managed.over_no_get  # 同上

In [24]:
obj.over_no_get = 7  # 触发描述符的 __set__ 方法，没有修改属性

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [25]:
obj.over_no_get  # 获取的仍然是托管类中的描述符实例

In [26]:
obj.__dict__['over_no_get'] = 9  # 通过实例的 __dict__ 属性设置实例属性
obj.over_no_get  # 实例属性会覆盖描述符，但只有读操作这样

9

In [27]:
obj.over_no_get = 7  # 经过描述符的 __set__ 方法处理

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [28]:
obj.over_no_get  # 读取时，只要有同名的实例属性，描述符就会被覆盖

9

没有实现 \_\_set__ 方法的描述符是非覆盖型描述符。

覆盖型描述符也叫做数据描述符或强制描述符；非覆盖型描述符也叫非数据描述符或遮盖型描述符。

In [29]:
obj = Managed()
obj.non_over  # 触发描述符的 __get__ 方法，第二个参数的值是 obj

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [30]:
obj.non_over = 7  # 非覆盖型描述符，没有干涉赋值操作的 __set__ 方法
obj.non_over  # 覆盖 Managed 类的同名描述符属性

7

In [31]:
Managed.non_over  # 描述符通过类截获访问

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [32]:
del obj.non_over  # 删除实例属性
obj.non_over  # 触发类中描述符的 __get__ 方法，第二个参数的值是托管实例

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


为类属性赋值都能覆盖描述符，这是一种猴子补丁技术。

读类属性的操作可以由依附在托管类上定义有 \_\_get__ 方法的描述符处理，但是写类属性的操作不会由依附在托管类上定义有 \_\_set__ 方法的描述符处理。

若想控制设置类属性的操作，要把描述符依附在类的类上，即依附在元类上。

In [33]:
obj = Managed()  # 创建实例
Managed.over = 1  # 覆盖类中的描述符属性
Managed.over_no_get = 2
Managed.non_over = 3
obj.over, obj.over_no_get, obj.non_over  # 没有描述符

(1, 2, 3)

在类中定义的函数属于绑定方法（bound method），方法是非覆盖型描述符。

In [34]:
obj = Managed()
obj.spam  # 获取的是绑定方法对象

<bound method Managed.spam of <__main__.Managed object at 0x00000233AA1F1160>>

In [35]:
Managed.spam  # 获取的是函数

<function __main__.Managed.spam(self)>

In [36]:
obj.spam = 7  # 覆盖类属性
obj.spam

7

通过托管类访问时，函数的 \_\_get__ 方法会返回自身的引用。通过实例访问时，函数的 \_\_get__ 方法返回的是绑定方法对象：一种可调用的对象，里面包装着函数，并把托管实例（例如 obj）绑定给函数的第一个参数（即 self）。

In [37]:
import collections


class Text(collections.UserString):

    def __repr__(self):
        return 'Text({!r})'.format(self.data)

    def reverse(self):
        return self[::-1]

In [38]:
word = Text('forward')
word  # Text 实例的 repr 方法返回一个类似 Text 构造方法调用的字符串，可用于创建相同的实例

Text('forward')

In [39]:
word.reverse()  # 返回反向的字符串

Text('drawrof')

In [40]:
Text.reverse(Text('backward'))  # 在类上调用方法相当于调用函数

Text('drawkcab')

In [41]:
type(Text.reverse), type(word.reverse)  # 类型不同

(function, method)

In [42]:
list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')]))  # Text.reverse 相当于函数

['diaper', (30, 20, 10), Text('desserts')]

In [43]:
Text.reverse.__get__(word)  # 函数都是非覆盖型描述符

<bound method Text.reverse of Text('forward')>

In [44]:
Text.reverse.__get__(None, Text)  # 调用函数的 __get__ 方法时，如果 instance 参数的值是 None ，那么得到的是函数本身

<function __main__.Text.reverse(self)>

In [45]:
word.reverse  # 调用 Text.reverse.__get__(word)，返回对应的绑定方法

<bound method Text.reverse of Text('forward')>

In [46]:
word.reverse.__self__  # 绑定方法对象有个 __self__ 属性，其值是调用这个方法的实例引用

Text('forward')

In [47]:
word.reverse.__func__ is Text.reverse  # 绑定方法的 __func__ 属性是依附在托管类上那个原始函数的引用

True

描述符用法建议
- 使用特性以保持简单
- 只读描述符必须有 \_\_set__ 方法
    - 否则，实例的同名属性会覆盖描述符
- 用于验证的描述符可以只有 \_\_set__ 方法
- 仅有 \_\_get__ 方法的描述符也可以实现高效缓存
- 非特殊的方法可以被实例属性覆盖
    - 解释器只会在类中寻找特殊的方法

描述符类的文档字符串用于注释托管类中的各个描述符实例。

在描述符类中，实现常规的 \_\_get__ 和（或）\_\_set__ 方法之外，可以实现 \_\_delete__ 方法，或者只实现 \_\_delete__ 方法就能实现托管属性的删除。

描述符类的关键优势：通过子类共享代码，构建具有部分相同功能的专用描述符。

非覆盖型描述符的一种具体类型：方法